**Author :  Ong Cheng Kei TP055620** <br>
**Description :**
<br>This file contains code to generate text embeddings for similarity calculation of ingredients name in two database : Nutrition5k, USDA-FNDDS.<br>This module exposes some function that can be called to get the most similar ingredient in both database

In [1]:
from pathlib import Path
from pprint import pprint
from types import SimpleNamespace

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from absl import logging
from tensorflow import keras

In [2]:
# Setting root directory to FoodNet
dir_parents = Path.cwd().parents
x = 0
root_dir = dir_parents[x]
while root_dir.name != "FoodNet" and x < len(dir_parents):
    x += 1
    root_dir = dir_parents[x]
assert (
    root_dir.name == "FoodNet"
), "Unable to find FoodNet root directory. Please change the root directory or set the working directory under the FoodNet root directory."

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)


def embed(input):
    return model(input)


if __name__ == "__main__":
    print("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [3]:
fndds_dir = root_dir / "Food Datasets" / "USDA-FNDDS"
with open((fndds_dir / "cleaned_food_category.txt").resolve(strict=True), "r") as file:
    fndds_category = file.read().split("\n")
    fndds_category.pop(-1)  # remove last line (empty)

In [4]:
if __name__ == "__main__":
    pprint(fndds_category)

['dips and gravies and other sauces',
 'meat mixed dishes',
 'tomato-based pasta sauces',
 'nonfat flavored milk',
 'egg rolls and dumplings and sushi',
 'mustard and other condiments',
 'cheese sandwiches',
 'pretzels or snack mix',
 'ice cream and frozen dairy desserts',
 'vegetable juice',
 'tortilla and corn and other chips',
 'other mexican mixed dishes',
 'candy not containing chocolate',
 'cream cheese and sour cream and whipped cream',
 'soy-based condiments',
 'olives and pickles and pickled vegetables',
 'fish',
 'reduced fat milk',
 'lowfat milk',
 'milk substitutes',
 'bananas',
 'poultry mixed dishes',
 'burritos and tacos',
 'pasta and noodles and cooked grains',
 'salad dressings and vegetable oils',
 'fried vegetables',
 'soft drinks',
 'beef but excludes ground',
 'gelatins and ices and sorbets',
 'sausages',
 'cabbage',
 'cakes and pies',
 'processed soy products',
 'chicken in whole pieces',
 'nonfat milk',
 'cheese',
 'corn',
 'macaroni and cheese',
 'margarine',
 '

In [5]:
df_fndds_nutrient_values = pd.read_csv(
    (fndds_dir / "cleaned_fndds_nutrient_values.csv").resolve(strict=True), sep="\t"
)

In [6]:
if __name__ == "__main__":
    display(df_fndds_nutrient_values)

,Main food description,WWEIA Category description,Energy (kcal),Protein (g),Carbohydrate (g),Total Fat (g)
0,"milk, not further specified",reduced fat milk,0.51,0.0334,0.0487,0.0199
1,"milk, whole",whole milk,0.60,0.0328,0.0467,0.0320
2,"milk, low sodium, whole",whole milk,0.61,0.0310,0.0446,0.0346
3,"milk, calcium fortified, whole",whole milk,0.60,0.0328,0.0467,0.0320
4,"milk, calcium fortified, low fat",lowfat milk,0.43,0.0338,0.0519,0.0095
...,...,...,...,...,...,...
6085,gin,liquor and cocktails,2.31,0.0000,0.0000,0.0000
6086,rum,liquor and cocktails,2.31,0.0000,0.0000,0.0000
6087,rum cooler,liquor and cocktails,0.68,0.0000,0.1007,0.0000
6088,vodka,liquor and cocktails,2.31,0.0000,0.0000,0.0000


In [7]:
nutrition5k_dir = root_dir / "Food Datasets" / "nutrition5k"
df_nutrition5k_nutrient_values = pd.read_csv(
    (nutrition5k_dir / "metadata" / "ingredients_metadata.csv").resolve(strict=True)
)

In [8]:
if __name__ == "__main__":
    display(df_nutrition5k_nutrient_values.head())

,ingr,id,cal/g,fat(g),carb(g),protein(g)
0,cottage cheese,1,0.980,0.043,0.034,0.110
1,strawberries,2,0.330,0.003,0.080,0.007
2,garden salad,3,0.646,0.034,0.032,0.061
3,bacon,4,5.410,0.420,0.014,0.370
4,potatoes,5,0.770,0.001,0.170,0.020


In [9]:
fndds_category_embeddings = embed(fndds_category)

In [10]:
fndds_description_embeddings = {}
fndds_description = {}
for category in fndds_category:
    all_food_in_category = df_fndds_nutrient_values[
        df_fndds_nutrient_values["WWEIA Category description"] == category
    ]["Main food description"].tolist()
    fndds_description_embeddings[category] = embed(all_food_in_category)
    fndds_description[category] = all_food_in_category

In [11]:
nutrition5k_ingredient = df_nutrition5k_nutrient_values["ingr"].tolist()
nutrition5k_ingredient_embeddings = embed(nutrition5k_ingredient)

In [12]:
if __name__ == "__main__":
    print("Below is an overview of Nutrition5k ingredient embeddings\n")
    pprint(nutrition5k_ingredient_embeddings)

Below is an overview of Nutrition5k ingredient embeddings

<tf.Tensor: shape=(555, 512), dtype=float32, numpy=
array([[-0.04213651,  0.02428615,  0.03386059, ...,  0.02607825,
         0.02512663, -0.05266538],
       [ 0.01148291,  0.00819834,  0.03557928, ...,  0.00613294,
         0.03251711, -0.06750827],
       [-0.01347014, -0.01774764,  0.01833388, ..., -0.04396154,
         0.0723353 , -0.0649578 ],
       ...,
       [ 0.04466048,  0.01656266,  0.02949798, ...,  0.00699597,
        -0.03023825, -0.07513158],
       [ 0.03270737, -0.01751684,  0.00517269, ..., -0.03976529,
        -0.01495524, -0.07337392],
       [-0.05237373, -0.04956251,  0.03297004, ..., -0.04588483,
        -0.04796786, -0.0534832 ]], dtype=float32)>


In [13]:
if __name__ == "__main__":
    print("Below is an overview of FNDDS category embeddings\n")
    pprint(fndds_category_embeddings)

Below is an overview of FNDDS category embeddings

<tf.Tensor: shape=(134, 512), dtype=float32, numpy=
array([[ 0.01735284,  0.02879251,  0.00810903, ...,  0.06208159,
        -0.02729057, -0.06811975],
       [-0.00028645,  0.01077796,  0.06082866, ...,  0.00663662,
        -0.01692836, -0.07342004],
       [ 0.00675197,  0.02172385,  0.05371361, ...,  0.02648405,
        -0.01341142, -0.07908035],
       ...,
       [ 0.03930959,  0.06037006, -0.04904855, ..., -0.01325004,
         0.01978845, -0.0774643 ],
       [-0.00472134,  0.06525755, -0.00535268, ..., -0.00607829,
        -0.02708187, -0.05188436],
       [ 0.02812375, -0.00056034,  0.05517928, ..., -0.03902761,
         0.05773354, -0.0327159 ]], dtype=float32)>


In [14]:
def get_cosine_similarity(matrix_embedding, target_vector_embedding):
    # compute the inner product
    similarity = tf.linalg.matvec(matrix_embedding, target_vector_embedding)
    return similarity


def get_most_similar_from_nutrition5k(target_vector_embedding):
    similarity = get_cosine_similarity(
        nutrition5k_ingredient_embeddings, target_vector_embedding
    )
    value, index = tf.math.top_k(similarity, k=1)
    value = tf.reshape(
        value,
        [
            1,
        ],
    )[0].numpy()
    index = tf.reshape(
        index,
        [
            1,
        ],
    )[0].numpy()
    return (value, index)


def get_most_similar_from_fndds(target_vector_embedding):
    category_similarity = get_cosine_similarity(
        fndds_category_embeddings, target_vector_embedding
    )
    category_value, category_index = tf.math.top_k(category_similarity, k=1)
    category_index = tf.reshape(
        category_index,
        [
            1,
        ],
    )[0].numpy()
    category = get_category_from_fndds(category_index)
    similarity = get_cosine_similarity(
        fndds_description_embeddings[category], target_vector_embedding
    )
    value, index = tf.math.top_k(similarity, k=1)
    value = tf.reshape(
        value,
        [
            1,
        ],
    )[0].numpy()
    index = tf.reshape(
        index,
        [
            1,
        ],
    )[0].numpy()
    return (value, index, category)


def get_ingredient_nutrient_from_nutrition5k(index):
    ingredient_name = nutrition5k_ingredient[index]
    result = df_nutrition5k_nutrient_values.loc[
        df_nutrition5k_nutrient_values["ingr"] == ingredient_name, :
    ]
    return result.squeeze()


def get_category_from_fndds(index):
    return fndds_category[index]


def get_ingredient_nutrient_from_fndds(category, index):
    ingredient_name = fndds_description[category][index]
    result = df_fndds_nutrient_values.loc[
        df_fndds_nutrient_values["Main food description"] == ingredient_name, :
    ]
    return result.squeeze()

In [15]:
exported = {
    fn.__name__: fn
    for fn in [
        get_ingredient_nutrient_from_fndds,
        get_ingredient_nutrient_from_nutrition5k,
        get_most_similar_from_fndds,
        get_most_similar_from_nutrition5k,
        embed,
    ]
}

In [16]:
exported = SimpleNamespace(**exported)

In [17]:
if __name__ != "__main__":
    print("Module ingredient_embeddings_similarity.ipynb is loaded")